In [10]:
import os
import sys
# sys.path.append(os.path.abspath("../src"))
sys.path.append("../src")

In [ ]:
import os
import sys
import requests
import re

from tqdm.autonotebook import tqdm, trange

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from tavily import TavilyClient

# os.chdir("../src/vectordb")
print(os.getcwd())
from vectordb.create_vectordb import PinconeVectorDb

from dotenv import load_dotenv, find_dotenv

# Load environment variables from.env file
load_dotenv(find_dotenv(), override=True)

C:\Users\kamran\AppData\Local\Temp\ipykernel_24692\2975920699.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


e:\Github Repositories\Research-Assistant-LangGraph\src\vectordb


True

In [15]:
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en"
EMBEDDING_DIMS = 384

In [ ]:
model_name = EMBEDDING_MODEL_NAME

embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [17]:
# Specify the arXiv ID for the paper
arxiv_id = '1706.03762'

# Make a GET request to retrieve the page for the specified paper
res = requests.get(f'https://arxiv.org/abs/{arxiv_id}')

# Access the content of the response as a string (HTML)
res.text

'<?xml version="1.0" encoding="UTF-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n\n<head>  <title>[1706.03762] Attention Is All You Need</title>\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n  <link rel="apple-touch-icon" sizes="180x180" href="/static/browse/0.3.4/images/icons/apple-touch-icon.png">\n  <link rel="icon" type="image/png" sizes="32x32" href="/static/browse/0.3.4/images/icons/favicon-32x32.png">\n  <link rel="icon" type="image/png" sizes="16x16" href="/static/browse/0.3.4/images/icons/favicon-16x16.png">\n  <link rel="manifest" href="/static/browse/0.3.4/images/icons/site.webmanifest">\n  <link rel="mask-icon" href="/static/browse/0.3.4/images/icons/safari-pinned-tab.svg" color="#5bbad5">\n  <meta name="msapplication-TileColor" content="#da532c">\n  <meta name="theme-color" content="#ffffff">\n  <

In [18]:
# Compile a regular expression pattern to find the abstract in the HTML response
abstract_pattern = re.compile(
    r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
    re.DOTALL
)

# Search for the abstract in the HTML response text
re_match = abstract_pattern.search(res.text)

# Check if the abstract was found and print it; otherwise, display an error message
if re_match:
    print(re_match.group(1))
else:
    print('Abstract not found.')

The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transforme

## Fetch Abstract from Arxiv using arxiv_id

In [19]:
abstract_pattern = re.compile(
    r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
    re.DOTALL
)

@tool
def fetch_abstract_from_arxiv(arxiv_id: str) -> str:
    """
    Fetches the abstract of the paper with the specified arXiv ID.
    Args:
        arxiv_id (str): The arXiv ID of the paper.
    Returns:
        str: The abstract of the paper, or an error message if not found.
    """

    res = requests.get(f'https://arxiv.org/abs/{arxiv_id}')
    
    re_match = abstract_pattern.search(res.text)

    return re_match.group(1) if re_match else 'Abstract not found.'


In [20]:
abstract = fetch_abstract_from_arxiv.invoke(input={"arxiv_id": arxiv_id})
abstract

'The dominant sequence transduction models are based on complex recurrent or convolutional neural networks in an encoder-decoder configuration. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the training costs of the best models from the literature. We show that the Transform

## RAG Search Tools

In [ ]:
def get_embeddings(texts: list[str],
                       model_name: str=EMBEDDING_MODEL_NAME):
        # Define the Hugging Face Embeddings
        embeddings = HuggingFaceEmbeddings(model_name=model_name)

        embeddings_list = []
        for text in texts:
            embeddings_list.append(embeddings.embed_query(text))

        return embeddings_list

In [21]:
def format_rag_results(matches: list) -> str:
    """
    Formats the RAG search results into a readable format.
    Args:
        matches (list): A list of matches from the RAG search.
    Returns:
        str: A formatted string containing the title, chunk, and arXiv ID of each match.
    """
    formatted_results = []
    
    # Loop through each match and extract its metadata.
    for match in matches:
        text = (
            f"Title: {match['metadata']['title']}\n"
            f"Chunk: {match['metadata']['chunk']}\n"
            f"ArXiv ID: {match['metadata']['arxiv_id']}\n"
        )
        # Append each formatted string to the results list.
        formatted_results.append(text)
    
    # Join all the individual formatted strings into one large string.
    return '\n---\n'.join(formatted_results)


In [22]:
pc = PinconeVectorDb()

In [24]:
pc.pc

In [ ]:
@tool
def rag_search_filter(query: str, arxiv_id: str) -> str:
    '''Finds information from the ArXiv database using a natural language query and a specific ArXiv ID.

    Args:
        query (str): The search query in natural language.
        arxiv_id (str): The ArXiv ID of the specific paper to filter by.
    
    Returns:
        str: A formatted string of relevant document contexts.
    '''
    
    # Encode the query into a vector representation.
    xq = get_embeddings(query)
    
    # Perform a search on the Pinecone index, filtering by ArXiv ID.
    xc = pc.pc.query(vector=xq, top_k=6, include_metadata=True, filter={'arxiv_id': arxiv_id})
    
    # Format and return the search results.
    return format_rag_results(xc['matches'])

In [ ]:
@tool
def rag_search(query: str) -> str:
    """
    Finds information from the ArXiv database using a natural language query.
    Args:
        query (str): The search query in natural language.
    Returns:
        str: A formatted string of relevant document contexts.
    """
    # Encode the query into a vector representation.
    xq = get_embeddings(query)
    
    # Perform a broader search without filtering by ArXiv ID.
    xc = pc.pc.query(vector=xq, top_k=5, include_metadata=True)
    
    # Format and return the search results.
    return format_rag_results(xc['matches'])


## Web Search Tool (Tavily)

In [ ]:
query = "where were olymics 2024 held?"
search = TavilyClient().search(query=query,
                                search_depth='advanced',
                                max_results=5)
results = search.get('results', [])

formatted_results = '\n---\n'.join(
    ['\n'.join([x['title'], x['content'], x['url']]) for x in results]
)
print(formatted_results)

2024 Olympics: Summer Games Dates, Logo, Location and Schedule
Paris 2024 Olympics. Watch more top videos, highlights, and B/R original content ... softball and karate—will not be held in Paris. ... As the 1900 Paris Games were the first to allow women to
https://bleacherreport.com/articles/2946974-2024-olympics-summer-games-dates-logo-location-and-schedule
---
2024 Olympics locations, venues, host cities - FOX Sports
2024 Summer Olympics Host Cities. Paris is the main host city for the 2024 Summer Olympic games, with events held at 16 other cities spread across Metropolitan France as well as one subsite in
https://www.foxsports.com/stories/olympics/2024-summer-olympics-locations-venues-host-cities
---
2024 Summer Olympics FAQ: Dates, locations, new sports, more
How many events were held at the 2024 Summer Olympics? The 2024 Paris Olympics featured 329 medal events, second most in the history of the Games (339 at the 2020 Tokyo Summer Olympics). Of the
https://www.espn.com/olympics/sto

In [ ]:
@tool
def web_search(query: str) -> str:
    """
    Finds general knowledge information using a Tavily search.
    Args:
        query (str): The search query string.
    
    Returns:
        str: A formatted string of the top search results, including title, content, and url.
    """
    
    search = TavilyClient().search(query=query,
                                search_depth='advanced',
                                max_results=5)
    results = search.get('results', [])

    formatted_results = '\n---\n'.join(
        ['\n'.join([x['title'], x['content'], x['url']]) for x in results]
    )
    
    # Return the formatted results or a 'No results found.' message if no results exist.
    return formatted_results if results else 'No results found.'

## Final Answer Tool

In [28]:
@tool
def final_answer(introduction: str,
                 research_steps: str | list,
                 main_body: str,
                 conclusion: str,
                 sources: str | list) -> str:
    """
    Constructs a final research report from the provided introduction, research steps, main body, conclusion, and sources.
    Args:
        introduction (str): A brief introduction to the research report.
        research_steps (str | list): The steps taken in the research process, either as a string or a list of strings.
        main_body (str): The main body of the research report.
        conclusion (str): The conclusion drawn from the research process.
        sources (str | list): The sources used in the research, either as a string or a list of strings.
    Returns:
        str: The final research report as a string.
    """

    # Format research steps if given as a list.
    if isinstance(research_steps, list):
        research_steps = '\n'.join([f'- {r}' for r in research_steps])
    
    # Format sources if given as a list.
    if isinstance(sources, list):
        sources = '\n'.join([f'- {s}' for s in sources])
    
    # Construct and return the final research report.
    return f'{introduction}\n\n \
        Research Steps:\n{research_steps}\n\n \
            Main Body:\n{main_body}\n\n \
                Conclusion:\n{conclusion}\n\n \
                    Sources:\n{sources}'
